# Neural network for the MNIST dataset
The MNIST dataset is a dataset of handwritten digits. Each digit is a 28x28 monochrome picture. 

In [37]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import random as rnd
import numpy.typing as npt

L1_SIZE = 56
L2_SIZE = 28
IMAGE_EDGE_SIZE = 28
PIXELS_PER_IMAGE = IMAGE_EDGE_SIZE ** 2
CLASSES_COUNT = 10
ITERATIONS = 500
LEARNING_RATE = 0.15

### Helper functions

In [38]:
def one_hot(labels: npt.ArrayLike) -> npt.NDArray:
    """
    Converts a 1D array of labels (the ground truth) to 2D matrix of shape (10, labels.size) as a probability distribution, 
    where the corresponding row given by the label value has probability of 1.
    
    :labels: The ground truth.
    :return: Encoded values of labels to probability distribution.
    """
    one_hot = np.zeros((10, labels.size))
    one_hot[labels, np.arange(labels.size)] = 1
    return one_hot

def get_accuracy(results: npt.NDArray, labels: npt.ArrayLike) -> float:
    """
    Calculates the accuracy of a neural network from the results of classification by comparing it to the ground truth.

    :results: The forward propagation results.
    :labels: The ground truth.
    :return: The accuracy as a real number. 
    """
    return (np.sum(np.argmax(results, 0) == labels) / labels.size)

def show_some_mistakes(results: npt.NDArray, labels: npt.ArrayLike, data: npt.NDArray, samples = 10) -> None:
    """
    Plots randomly choosen images, which were not classified correctly.

    :results: The forward propagation results.
    :labels: The ground truth.
    :data: The input data of forward propagation, i.e images.
    :samples: The number of shown images, 10 by default.
    """
    results = np.argmax(results, 0)
    i = rnd.randint(0, labels.size)
    j = 0
    while j < samples:
        i = (i + 1) % labels.size
        if results[i] != labels[i]:
            print("labeled:", labels[i], "-- classified:", results[i])
            plt.imshow(data[:, i].reshape((IMAGE_EDGE_SIZE, IMAGE_EDGE_SIZE)), cmap='gray')
            plt.show()
            j += 1

### Non linear functions

In [39]:
def ReLU(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the Rectified Linear Units of a numpy matrix.
    
    :L: Matrix of values of a hidden layer.
    :return: For all nonnegative numbers returns its value, otherwise 0.
    """
    return np.maximum(0, L)

def ReLU_deriv(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the derivation of ReLu function of a numpy matrix.

    :L: Matrix of values of a hidden layer.
    :return: For all positive numbers returns 1, otherwise 0.
    """
    return L > 0

def sigmoid(L: npt.NDArray) -> npt.NDArray:
    """
    Calculates the Sigmoid function of a numpy matrix.
    
    :L: Values of a hidden layer.
    :return: For all indexes with value x returns 1 / (1 + e^(-x)).
    """
    return 1 / (1 + np.exp(-L))

def softmax(L: npt.NDArray) -> npt.NDArray:
    """
    Converts matrix of N values in a row to probability distribution of N outcomes for each row.

    :L: Values of an output layer.
    :return: For all indexes of the given matrix returns the probability of a given index in its row.
    """
    return np.exp(L) / sum(np.exp(L))

### Initialization of weights and biases

In [40]:
def weights_init_2L(size: int, input_size = PIXELS_PER_IMAGE, output_size = CLASSES_COUNT) -> tuple:
    """
    Randomly initilizes weights with real numbers between -0.5 and 0.5.

    :size: Number of neurons in a hidden layer.
    :input_size: Number of input values.
    :output_size: Number of neurons in a hidden layer an output layer.
    :return: Tuple of randomly initilized weight matrices real numbers between -0.5 and 0.5.
    """
    # The order of matrix multiplication by weight matrices is given by the arrangement of weights into columns and rows, see below and in other functions.
    W1 = np.random.rand(size, input_size) - 0.5  # Each column is applied to respective input value, weights for each neuron of the first hidden layer are in rows.
    W2 = np.random.rand(output_size, size) - 0.5 # Each column is applied to respective neuron of the first hidden layer, weights for each neuron of the output layer are in rows.
    return W1, W2

def weights_init_3L(l1_size: int, l2_size: int, input_size = PIXELS_PER_IMAGE, output_size = CLASSES_COUNT) -> tuple:
    """
    Randomly initilizes weights with real numbers between -0.5 and 0.5.

    :l1_size: Number of neurons in a first hidden layer.
    :l2_size: Number of neurons in a second hidden layer.
    :input_size: Number of input values.
    :output_size: Number of neurons in an output layer.
    :return: Tuple of randomly initilized weight matrices with real numbers between -0.5 and 0.5.
    """
    # The order of matrix multiplication by weight matrices is given by the arrangement of weights into columns and rows, see below and in other functions.
    W1 = np.random.rand(l1_size, input_size) - 0.5   # Each column is applied to respective input value, weights for each neuron of the first hidden layer are in rows.
    W2 = np.random.rand(l2_size, l1_size) - 0.5      # Each column is applied to respective neuron of the first hidden layer, weights for each neuron of the second hidden layer are in rows.
    W3 = np.random.rand(output_size, l2_size) - 0.5  # Each column is applied to respective neuron of the second hidden layer, weights for each neuron of the out layer are in rows.
    return W1, W2, W3

def weights_init(sizes: list) -> list:
    """
    Randomly initilizes weights with real numbers between -0.5 and 0.5.

    :sizes: List of layer sizes, first value specifies number of input values, last value specifies number of output neurons. 
            Number of initilizes weight matrices is one less than lenght of the given list.
    :return: List of randomly initilized weight matrices with real numbers between -0.5 and 0.5.
    """
    weights = [] 
    for i in range(len(sizes) - 1):
        # The order of matrix multiplication by weight matrices is given by the arrangement of weights into columns and rows, see below and in other functions.
        weights.append(np.random.rand(sizes[i + 1], sizes[i]) - 0.5) # Columns represent weights for Nth layer values, rows represent weights for N+1th layer values.
    return weights

def biases_init_2L(size: int, output_size = 10) -> tuple:
    """
    Randomly initilizes biases with real numbers between -0.5 and 0.5.

    :size: Number of neurons in a hidden layer.
    :output_size: Number of neurons in an output layer.
    :return: Tuple of randomly initilized bias vectors with real numbers between -0.5 and 0.5.
    """
    # The biases are column vectors
    B1 = np.random.rand(size, 1) - 0.5          # One bias for each neuron in the hidden layer.
    B2 = np.random.rand(output_size, 1) - 0.5   # One bias for each neuron in the output layer.
    return B1, B2

def biases_init_3L(l1_size, l2_size, output_size = CLASSES_COUNT):
    """
    Randomly initilizes biases with real numbers between -0.5 and 0.5.

    :l1_size: Number of neurons in a first hidden layer.
    :l2_size: Number of neurons in a second hidden layer.
    :output_size: Number of neurons in an output layer.
    :return: Tuple of randomly initilized bias vectors with real numbers between -0.5 and 0.5.
    """
    # The biases are column vectors
    B1 = np.random.rand(l1_size, 1) - 0.5       # One bias for each neuron in the first hidden layer.
    B2 = np.random.rand(l2_size, 1) - 0.5       # One bias for each neuron in the second hidden layer.
    B3 = np.random.rand(output_size, 1) - 0.5   # One bias for each neuron in the output layer.
    return B1, B2, B3

def biases_init(sizes):
    biases = []
    for i in range(1, len(sizes)): # skips the input valies size
        biases.append(np.random.rand(sizes[i], 1) - 0.5)    # Column vectors for each layer.
    return biases

### Adjusting weights and biases after a training cycle

In [41]:
def weights_adjust_2L(W1: npt.NDArray, W2: npt.NDArray, dW1: npt.NDArray, dW2: npt.NDArray, learning_rate: float) -> tuple:
    """
    Adjusts weights according to the gradients and learning rate.

    :W1: First layer weight matrix.
    :W2: Second layer weight matrix.
    :dW1: First layer gradient matrix.
    :dW2: Second layer gradient matrix.
    :return: Tuple of adjusted weight matrices.
    """
    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    return W1, W2

def weights_adjust_3L(W1: npt.NDArray, W2: npt.NDArray, W3: npt.NDArray, dW1: npt.NDArray, dW2: npt.NDArray, dW3: npt.NDArray, learning_rate: float) -> tuple:
    """
    Adjusts weights according to the gradients and learning rate.

    :W1: First layer weight matrix.
    :W2: Second layer weight matrix.
    :W2: Third layer weight matrix.
    :dW1: First layer gradient matrix.
    :dW2: Second layer gradient matrix.
    :dW2: Third layer gradient matrix.
    :return: Tuple of adjusted weight matrices.
    """
    W1 = W1 - learning_rate * dW1
    W2 = W2 - learning_rate * dW2
    W3 = W3 - learning_rate * dW3
    return W1, W2, W3

def weights_adjust(weights: list, weight_gradients: list, learning_rate: float) -> list:
    """
    Adjusts weights according to the gradients and learning rate.

    :weights: List of weight matrices.
    :weight_gradients: List of gradient matrices of the same shape as respective weight matrices.
    :return: List of adjusted weight matrices.
    """
    for i in range(len(weights)):
        weights[i] = weights[i] - learning_rate * weight_gradients[i]
        
    return weights

def biases_adjust_2L(B1: npt.ArrayLike, B2: npt.ArrayLike, dB1: float, dB2: float, learning_rate: float) -> tuple:
    """
    Adjusts biases according to the gradients and learning rate.

    :B1: First layer bias column vector.
    :B2: Second layer bias column vector.
    :dB1: First layer bias difference (scalar value).
    :dB2: Second layer bias difference (scalar value).
    :return: Tuple of adjusted bias column vectors.
    """
    B1 = B1 - learning_rate * dB1
    B2 = B2 - learning_rate * dB2
    return B1, B2

def biases_adjust_3L(B1: npt.ArrayLike, B2: npt.ArrayLike, B3: npt.ArrayLike, dB1: float, dB2: float, dB3: float, learning_rate: float) -> tuple:
    """
    Adjusts biases according to the difference and learning rate.

    :B1: First layer bias column vector.
    :B2: Second layer bias column vector.
    :B2: Third layer bias column vector.
    :dB1: First layer bias difference (scalar value).
    :dB2: Second layer bias difference (scalar value).
    :dB2: Third layer bias difference (scalar value).
    :return: Tuple of adjusted bias column vectors.
    """
    B1 = B1 - learning_rate * dB1
    B2 = B2 - learning_rate * dB2
    B3 = B3 - learning_rate * dB3
    return B1, B2, B3

def biases_adjust(biases: npt.ArrayLike, bias_diff: npt.ArrayLike, learning_rate: float) -> list:
    """
    Adjusts biases according to the gradients and learning rate.

    :biases: List of bias column vectors.
    :bias_diff: List of bias differences (scalar values).
    :return: List of adjusted bias column vectors.
    """
    for i in range(len(biases)):
        biases[i] = biases[i] - learning_rate * bias_diff[i]
    
    return biases

### Forward propagation

In [42]:
def forward_prop_2L(W1: npt.NDArray, W2: npt.NDArray, B1: npt.ArrayLike, B2: npt.ArrayLike, training_data: npt.ArrayLike) -> tuple:
    """
    Classifies all training samples in to classes according to the given weights and biases.

    :W1: First layer weight matrix.
    :W2: Second layer weight matrix.
    :B1: First layer bias column vector.
    :B2: Second layer bias column vector.
    :training_data: Training samples, each column represents one training sample.
    :return: Tuple of matrices with values for each layer.
    """
    Z1 = W1.dot(training_data) + B1 # Matrix multiplication of the training samples and first layer weights with addition of biases. The calculated matrix shape is: rows = W1.rows, cols = training_data.cols.
    L1 = ReLU(Z1)                   # Nonlinear activation function applied on the first layer values. The shape of the resulting matrix is the stays the same.
    Z2 = W2.dot(L1) + B2            # Matrix multiplication of the first layer values and second layer weights with addition of biases. The calculated matrix shape is: rows = W2.rows, cols = training_data.cols.
    L2 = softmax(Z2)                # Transformation of the second layer outputs to a probability distribution. The shape of the resulting matrix is the stays the same.
    return L1, L2

def forward_prop_3L(W1: npt.NDArray, W2: npt.NDArray, W3: npt.NDArray, B1: npt.ArrayLike, B2: npt.ArrayLike, B3: npt.ArrayLike, training_data: npt.NDArray) -> tuple:
    """
    Classifies all training samples in to classes according to the given weights and biases.

    :W1: First layer weight matrix.
    :W2: Second layer weight matrix.
    :W3: Third layer weight matrix.
    :B1: First layer bias column vector.
    :B2: Second layer bias column vector.
    :B3: Third layer bias column vector.
    :training_data: Training samples, each column represents one training sample.
    :return: Tuple of matrices with values for each layer.
    """
    Z1 = W1.dot(training_data) + B1 # Matrix multiplication of the training samples and first layer weights with addition of biases. The calculated matrix shape is: rows = W1.rows, cols = training_data.cols.
    L1 = ReLU(Z1)                   # Nonlinear activation function applied on the first layer values. The shape of the resulting matrix is the stays the same.
    Z2 = W2.dot(L1) + B2            # Matrix multiplication of the first layer values and second layer weights with addition of biases. The calculated matrix shape is: rows = W2.rows, cols = training_data.cols.
    L2 = ReLU(Z2)                   # Nonlinear activation function applied on the second layer values. The shape of the resulting matrix is the stays the same.
    Z3 = W3.dot(L2) + B3            # Matrix multiplication of the second layer values and third layer weights with addition of biases. The calculated matrix shape is: rows = W3.rows, cols = training_data.cols.
    L3 = softmax(Z3)                # Transformation of the third layer outputs to a probability distribution. The shape of the resulting matrix is the stays the same.
    return L1, L2, L3

def forward_prop(weights: npt.NDArray, biases: npt.ArrayLike, activations: list, training_data: npt.NDArray) -> list:
    """
    Classifies all training samples in to classes according to the given weights and biases.

    :weights: List of weight matrices.
    :biases: List of bias column vectors.
    :activations: List of nonlinear activation functions.
    :training_data: Training samples, each column represents one training sample.
    :return: List of matrices with values for each layer.
    """
    layers = []
    input = training_data
    for i in range(len(weights)):
        layers.append(activations[i](weights[i].dot(input) + biases[i])) # Matrix multiplication of the Nth layer and weights of the N+1th layer with addition of biases and aplication of nonlinear activation function.
        input = layers[i]   # Save the current layer
    
    return layers

### Backward propagation

In [43]:
def back_prop_2L(L1: npt.NDArray, L2: npt.NDArray, W2: npt.NDArray, training_data: npt.NDArray, one_hot_labels: npt.NDArray) -> tuple:
    """
    Calculates weight and bias gradients for all layers.

    :L1: Matrix of first layer values.
    :L2: Matrix of second layer values.
    :W2: Matrix of second layer weights.
    :training_data: Training samples, each column represents one training sample.
    :one_hot_labels: Encoded ground truth labels to probabilty distribution.
    :return: Tuple of matrices with weight and bias gradients for each layer.
    """
    m = one_hot_labels.shape[1] # The number of training samples. 
    dZ2 = L2 - one_hot_labels   # Calculating errors with matrix subtractions between the classified values and the ground truth. Subtraction does not change the shape of the resulting matrix.
    dW2 = dZ2.dot(L1.T) / m     # Matrix multiplication of the output layer errors and hidden layer values normalized by the number of training samples. The calculated matrix shape is: rows = one_hot_labels.rows == W2.rows (10 in case of digit classification), cols = L1.rows == W2.cols (L1 is transposed).
    dB2 = np.sum(dZ2) / m       # Arithmetic average of the errors. The result is a scalar value.
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(L1) # Back propagating the errors from output layer to the hidden layer by matrix multiplication of the transponed weight matrix and errors multiplied by the ReLU derivation. The calculated matrix shape is: rows: W2.cols == W1.rows (W2 is transposed), cols = one_hot_labels.cols (number of training samples).
    dW1 = dZ1.dot(training_data.T) / m   # Matrix multiplication of the hidden layer errors and input values normalized by the number of training samples. The calculated matrix shape is: rows = dZ1.rows == W1.rows, cols = training_data.rows == W1.cols (training_data is transposed, in case of images 28x28 it is 784).
    dB1 = np.sum(dZ1) / m                # Arithmetic average of the hidden layer errors. The result is a scalar value.
    return dW1, dW2, dB1, dB2

def back_prop_3L(L1: npt.NDArray, L2: npt.NDArray, L3: npt.NDArray, W2: npt.NDArray, W3: npt.NDArray, training_data: npt.NDArray, one_hot_labels: npt.NDArray) -> tuple:
    """
    Calculates weight and bias gradients for all layers.

    :L1: Matrix of first layer values.
    :L2: Matrix of second layer values.
    :L3: Matrix of third layer values.
    :W2: Matrix of second layer weights.
    :W3: Matrix of third layer weights.
    :training_data: Training samples, each column represents one training sample.
    :one_hot_labels: Encoded ground truth labels to probabilty distribution.
    :return: Tuple of matrices with weight and bias gradients for each layer.
    """
    m = one_hot_labels.shape[1]  # The number of training samples.
    dZ3 = L3 - one_hot_labels    # Calculating errors with matrix subtractions between the classified values and the ground truth. Subtraction does not change the shape of the resulting matrix.
    dW3 = dZ3.dot(L2.T) / m      # Matrix multiplication of the output layer errors and hidden layer values normalized by the number of training samples. The calculated matrix shape is: rows = one_hot_labels.rows == W3.rows (10 in case of digit classification), cols = L2.rows == W3.cols (L2 is transposed).
    dB3 = np.sum(dZ3) / m        # Arithmetic average of the errors. The result is a scalar value.
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(L2) # Back propagating the errors from output layer to the last hidden layer by matrix multiplication of the transponed weight matrix and errors by the ReLU derivation. The calculated matrix shape is: rows: W3.cols == W2.rows (W3 is transposed), cols = one_hot_labels.cols (number of training samples).
    dW2 = dZ2.dot(L1.T) / m      # Matrix multiplication of the last hidden layer errors and first layer values normalized by the number of training samples. The calculated matrix shape is: rows = dZ2.rows == W2.rows, cols = L1.rows == W2.cols (L1 is transposed).
    dB2 = np.sum(dZ2) / m        # Arithmetic average of the errors. The result is a scalar value.
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(L1) # Back propagating the errors from last hidden layer to the first (second to last) hidden layer by matrix multiplication of the transponed weight matrix and errors by the ReLU derivation. The calculated matrix shape is: rows: W2.cols == W1.rows (W2 is transposed), cols = dZ2.cols (number of training samples).
    dW1 = dZ1.dot(training_data.T) / m   # Matrix multiplication of the first hidden layer errors and input values normalized by the number of training samples. The calculated matrix shape is: rows = dZ1.rows == W1.rows, cols = training_data.rows == W1.cols (training_data is transposed, in case of images 28x28 it is 784).
    dB1 = np.sum(dZ1) / m                # Arithmetic average of the errors. The result is a scalar value.
    return dW1, dW2, dW3, dB1, dB2, dB3

def back_prop(layers: list, weights: list, activation_derivs: list, training_data: npt.NDArray, one_hot_labels: npt.NDArray) -> tuple:
    """
    Calculates weight and bias gradients for all layers.

    :layers: List of layer matrices.
    :weights: List of weight matrices.
    :activation_derivs: List of derivations of nonlinear activation functions.
    :training_data: Training samples, each column represents one training sample.
    :one_hot_labels: Encoded ground truth labels to probabilty distribution.
    :return: Tuple of lists of matrices with weight and bias differences for each layer.
    """
    layers_count = len(layers)                  # The number of layers.
    m = one_hot_labels.shape[1]                 # The number of training samples.
    weight_gradients = [None] * layers_count    # Initialization of weight gradients list
    bias_diff = [None] * layers_count           # Initialization of bias differences list

    current_deriv = layers[layers_count - 1] - one_hot_labels          # Calculating errors with matrix subtractions between the classified values and the ground truth. Subtraction does not change the shape of the resulting matrix.
    for i in range(layers_count - 1, 0, -1):  # Ranging from N to 1
        weight_gradients[i] = current_deriv.dot(layers[i - 1].T) / m   # Matrix multiplication of the N+1th hidden layer errors and Nth hidden layer values normalized by the number of training samples. The shape of the resulting matrix changes with iterations, see above.
        bias_diff[i] = np.sum(current_deriv) / m                       # Arithmetic average of the errors - scalar value.
        current_deriv = weights[i].T.dot(current_deriv) * activation_derivs[i - 1](layers[i - 1]) # Back propagating the errors from N+1th layer to the Nth layer by matrix multiplication of the transponed weight matrix and errors by the derivation of activation function. The shape of the resulting matrix changes with iterations, see above.
    
    # Calculation for the first layer
    weight_gradients[0] = current_deriv.dot(training_data.T) / m    # Matrix multiplication of the first hidden layer errors and input values normalized by the number of training samples.
    bias_diff[0] = np.sum(current_deriv) / m                        # Arithmetic average of the errors.

    return weight_gradients, bias_diff

### Gradient descend

In [44]:
def gradient_descend_2L(training_data: npt.NDArray, labels: npt.ArrayLike, L1_size: int, iterations: int, learning_rate: float) -> tuple:
    """
    Trains a neural network with 1 hidden layer using gradient descend method.

    :training_data: Traning samples organized into columns.
    :labels: Ground truth values for each training sample.
    :L1_size: Number of neurons in the hidden neuron layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    :return: Tuple of weight and bias matrices.
    """
    # initialization
    W1, W2 = weights_init_2L(L1_size)
    B1, B2 = biases_init_2L(L1_size)
    labels = one_hot(labels)

    # training
    for _ in range(iterations):
        L1, L2 = forward_prop_2L(W1, W2, B1, B2, training_data)
        dW1, dW2, dB1, dB2 = back_prop_2L(L1, L2, W2, training_data, labels)
        W1, W2 = weights_adjust_2L(W1, W2, dW1, dW2, learning_rate)
        B1, B2 = biases_adjust_2L(B1, B2, dB1, dB2, learning_rate)

    return W1, W2, B1, B2

def gradient_descend_3L(training_data: npt.NDArray, labels: npt.ArrayLike, L1_size: int, L2_size: int, iterations: int, learning_rate: float) -> tuple:
    """
    Trains a neural network with 2 hidden layers using gradient descend method.

    :training_data: Traning samples organized into columns.
    :labels: Ground truth values for each training sample.
    :L1_size: Number of neurons in the first hidden neuron layer.
    :L2_size: Number of neurons in the second hidden neuron layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    :return: Tuple of weight and bias matrices.
    """
    # initialization
    W1, W2, W3 = weights_init_3L(L1_size, L2_size)
    B1, B2, B3 = biases_init_3L(L1_size, L2_size)
    labels = one_hot(labels)

    # training 
    for _ in range(iterations):
        L1, L2, L3 = forward_prop_3L(W1, W2, W3, B1, B2, B3, training_data)
        dW1, dW2, dW3, dB1, dB2, dB3 = back_prop_3L(L1, L2, L3, W2, W3, training_data, labels)
        W1, W2, W3 = weights_adjust_3L(W1, W2, W3, dW1, dW2, dW3, learning_rate)
        B1, B2, B3 = biases_adjust_3L(B1, B2, B3, dB1, dB2, dB3, learning_rate)

    return W1, W2, W3, B1, B2, B3

def gradient_descend(training_data: npt.NDArray, labels: npt.ArrayLike, weight_sizes: list, activations: list, activation_derivs: list, iterations: int, learning_rate: float) -> tuple:
    """
    Trains a neural network given number of layers using gradient descend method.

    :training_data: Traning samples organized into columns.
    :labels: Ground truth values for each training sample.
    :weigh_sizes: List of numbers of neurons in each layer including the input layer.
    :activations: List of activation functions for each hidden layer and output layer.
    :activation_derivs: List of derivation of activation functions each hidden layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    :return: Tuple of list of weight and bias matrices.
    """
    # initialization
    weights = weights_init(weight_sizes)
    biases = biases_init(weight_sizes)
    labels = one_hot(labels)

    # training
    for _ in range(iterations):
        layers = forward_prop(weights, biases, activations, training_data)
        weight_gradients, bias_diff = back_prop(layers, weights, activation_derivs, training_data, labels)
        weights = weights_adjust(weights, weight_gradients, learning_rate)
        biases = biases_adjust(biases, bias_diff, learning_rate)

    return weights, biases

### Training functions

In [45]:
def train_2L(training_data: npt.NDArray, training_labels: npt.ArrayLike, L1_size = L1_SIZE, iterations = ITERATIONS, learning_rate = LEARNING_RATE):
    """
    Trains a neural network with one hidden layer using gradient descend and saves the result.

    :training_data: Traning samples organized into columns.
    :training_labels: Ground truth values for each training sample.
    :L1_size: Number of neurons in the first hidden neuron layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    """
    # training
    W1, W2, B1, B2 = gradient_descend_2L(training_data, training_labels, L1_size, iterations, learning_rate)

    # saving results
    np.save("W1_2L.npy", W1)
    np.save("W2_2L.npy", W2)
    np.save("B1_2L.npy", B1)
    np.save("B2_2L.npy", B2)

def train_3L(training_data, training_labels, L1_size = L1_SIZE, L2_size = L2_SIZE, iterations = ITERATIONS, learning_rate = LEARNING_RATE):
    """
    Trains a neural network with two hidden layers using gradient descend and saves the result.

    :training_data: Traning samples organized into columns.
    :training_labels: Ground truth values for each training sample.
    :L1_size: Number of neurons in the first hidden neuron layer.
    :L2_size: Number of neurons in the second hidden neuron layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    """
    # training
    W1, W2, W3, B1, B2, B3 = gradient_descend_3L(training_data, training_labels, L1_size, L2_size, iterations, learning_rate)

    # saving results
    np.save("W1_3L.npy", W1)
    np.save("W2_3L.npy", W2)
    np.save("W3_3L.npy", W3)
    np.save("B1_3L.npy", B1)
    np.save("B2_3L.npy", B2)
    np.save("B3_3L.npy", B3)

def train(training_data: npt.NDArray, training_labels: npt.ArrayLike, weight_sizes: list, activations: list, activation_derivs: list, iterations = ITERATIONS, learning_rate = LEARNING_RATE, 
          files_id = dt.now().strftime("%d-%m-%Y_%H:%M:%S")):
    """
    Trains a neural network with a given number of hidden layers using gradient descend and saves the result.

    :training_data: Traning samples organized into columns.
    :labels: Ground truth values for each training sample.
    :weigh_sizes: List of numbers of neurons in each layer including the input layer.
    :activations: List of activation functions for each hidden layer and output layer.
    :activation_derivs: List of derivation of activation functions each hidden layer.
    :iterations: Number of training iterations.
    :learning_rate: Learning rate used to adjust weights and biases.
    :return: Tuple of list of weight and bias matrices.
    """
    # training
    weights, biases = gradient_descend(training_data, training_labels, weight_sizes, activations, activation_derivs, iterations, learning_rate)

    # saving results
    for i in range(len(weights)):
        j = str(i + 1)
        np.save("W" + j + "_" + files_id + ".npy", weights[i])
        np.save("B" + j + "_" + files_id + ".npy", biases[i])
    

### Funtions for loading of training and testing data

In [46]:
def load_training_data() -> tuple:
    """
    Loads training data and training labels from files and transforms them to desired shape.

    :return: Matrix of training data and array of training labels.
    """
    training_data = idx2numpy.convert_from_file("mnist/train-images.idx3-ubyte")
    training_labels = idx2numpy.convert_from_file("mnist/train-labels.idx1-ubyte")
    sample_count = training_data.shape[0]
    training_data = np.reshape(training_data, (sample_count, -1)).T / 255
    return training_data, training_labels

def load_test_data() -> tuple:
    """
    Loads testing data and training labels from files and transforms them to desired shape.

    :return: Matrix of testing data and array of testing labels.
    """
    test_data = idx2numpy.convert_from_file("mnist/t10k-images.idx3-ubyte")
    test_labels = idx2numpy.convert_from_file("mnist/t10k-labels.idx1-ubyte")
    sample_count = test_data.shape[0]
    test_data = np.reshape(test_data, (sample_count, -1)).T / 255
    return test_data, test_labels

### Functions for result assessment

In [47]:
def assess_results_2L():
    """
    Assesses the results of a neural network with one hidden layer.
    """
    # Loading of the trained weights and biases.
    W1 = np.load("W1_2L.npy")
    W2 = np.load("W2_2L.npy")
    B1 = np.load("B1_2L.npy")
    B2 = np.load("B2_2L.npy")
    print("\n###################### 2 layers results ######################\n")

    training_data, training_labels = load_training_data()
    _, L2 = forward_prop_2L(W1, W2, B1, B2, training_data)                 # Calculation of the network output for each training sample.
    print("Accuracy on training set: ", get_accuracy(L2, training_labels)) # Calculation of the accuracy of the network on training data.

    test_data, test_labels = load_test_data()
    _, L2 = forward_prop_2L(W1, W2, B1, B2, test_data)             # Calculation of the network output for each testing sample.
    print("Accuracy on test set: ", get_accuracy(L2, test_labels)) # Calculation of the accuracy of the network on training data.

    show_some_mistakes(L2, test_labels, test_data)

def assess_results_3L():
    """
    Assesses the results of a neural network with two hidden layer.
    """
    # Loading of the trained weights and biases.
    W1 = np.load("W1_3L.npy")
    W2 = np.load("W2_3L.npy")
    W3 = np.load("W3_3L.npy")
    B1 = np.load("B1_3L.npy")
    B2 = np.load("B2_3L.npy")
    B3 = np.load("B3_3L.npy")
    print("\n###################### 3 layers results ######################\n")

    training_data, training_labels = load_training_data()
    _, _, L3 = forward_prop_3L(W1, W2, W3, B1, B2, B3, training_data)      # Calculation of the network output for each training sample.
    print("Accuracy on training set: ", get_accuracy(L3, training_labels)) # Calculation of the accuracy of the network on training data.

    test_data, test_labels = load_test_data()
    _, _, L3 = forward_prop_3L(W1, W2, W3, B1, B2, B3, test_data)   # Calculation of the network output for each testing sample.
    print("Accuracy on test set: ", get_accuracy(L3, test_labels))  # Calculation of the accuracy of the network on training data.

    show_some_mistakes(L3, test_labels, test_data)

def assess_results(activations: list, files_id = ""):
    """
    Assesses the results of a neural network with given number of layers including the output layer.

    :activations: Activation functions for each hidden layer and for the output layer.
    :files_id: Id in the file names of the weights and biases.
    """
    layers_count = len(activations)
    weights = []
    biases = []
    # Loading of the trained weights and biases.
    for i in range(layers_count):
        j = str(i + 1)
        weights.append(np.load("W" + j + "_" + files_id + ".npy"))
        biases.append(np.load("B" + j + "_" + files_id + ".npy"))
    
    print("\n###################### " + str(layers_count) + " layers results ######################\n")

    training_data, training_labels = load_training_data()
    last_layer = forward_prop(weights, biases, activations, training_data)[layers_count - 1] # Calculation of the network output for each training sample.
    print("Accuracy on training set: ", get_accuracy(last_layer, training_labels))           # Calculation of the accuracy of the network on training data.

    test_data, test_labels = load_test_data()
    last_layer = forward_prop(weights, biases, activations, test_data)[layers_count - 1]     # Calculation of the network output for each testing sample.
    print("Accuracy on test set: ", get_accuracy(last_layer, test_labels))                   # Calculation of the accuracy of the network on training data.

    show_some_mistakes(last_layer, test_labels, test_data)

### Training

In [48]:
training_data, training_labels = load_training_data()
train_2L(training_data, training_labels)
train_3L(training_data, training_labels)
train(training_data, training_labels, [PIXELS_PER_IMAGE, 112, L1_SIZE, L2_SIZE, CLASSES_COUNT], [ReLU, ReLU, ReLU, softmax], 
      [ReLU_deriv, ReLU_deriv, ReLU_deriv], files_id="4L")

dB1  1.789861814668362 (60000,)
dB1  0.7747461507313812 (60000,)
dB1  0.5822687881156722 (60000,)
dB1  0.34759051712157424 (60000,)
dB1  0.2910259269995468 (60000,)
dB1  0.2181509372789305 (60000,)
dB1  0.17940231817778304 (60000,)
dB1  0.14057825353337874 (60000,)
dB1  0.11501808553553791 (60000,)
dB1  0.09303067971053905 (60000,)
dB1  0.0759513602197056 (60000,)
dB1  0.060137926127678605 (60000,)
dB1  0.04814020460852757 (60000,)
dB1  0.03719162472792257 (60000,)
dB1  0.028239929287058037 (60000,)
dB1  0.021397758088115695 (60000,)
dB1  0.015592130213674667 (60000,)
dB1  0.010625899911704312 (60000,)
dB1  0.006582425202308468 (60000,)
dB1  0.002669330028510969 (60000,)
dB1  -0.0007004206123804344 (60000,)
dB1  -0.0034943080270543295 (60000,)
dB1  -0.0058497706742344485 (60000,)
dB1  -0.007816654012766176 (60000,)
dB1  -0.0093308525144009 (60000,)
dB1  -0.010324456708920488 (60000,)
dB1  -0.011866930868549347 (60000,)
dB1  -0.012593715341314817 (60000,)
dB1  -0.013439079954007343 (600

KeyboardInterrupt: 

### Result assessment

In [ ]:
assess_results_2L()
assess_results_3L()
assess_results([ReLU, ReLU, ReLU, softmax], "4L")

In [ ]:
# TODO dropout
pKeep = 0.8
weights = np.ones([5, 5])
binary_value = np.random.rand(weights.shape[0], weights.shape[1]) < pKeep
res = np.multiply(weights, binary_value)